# quickvc
[![Package badge]][github]
[![Open In Colab]][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/QuickVC-official
[notebook]:https://colab.research.google.com/github/tarepan/QuickVC-official/blob/main/quickvc.ipynb
[tarepan]:https://github.com/terepan
[Package badge]:https://img.shields.io/badge/GitHub-quickvc-9cf.svg
[Open In Colab]:https://colab.research.google.com/assets/colab-badge.svg

## Colab Check
Check environments.

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}' # Google Colaboratory runnning time
!head -n 1 /proc/driver/nvidia/gpus/**/information                  # GPU type
!/usr/local/cuda/bin/nvcc --version | sed '4!d'                     # CUDA version
!python --version                                                   # Python version
!pip show torch | sed '2!d'                                         # PyTorch version

## Setup

Install the package from `terepan/QuickVC-official` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==2.0.0" -q            # Based on your PyTorch environment
# !pip install "torchaudio==2.0.1" -q       # Based on your PyTorch environment
# repository install
# !pip uninstall packname -y -q
# !pip install git+https://github.com/tarepan/QuickVC-official -q
!git clone https://github.com/tarepan/QuickVC-official.git
%cd QuickVC-official

In [ ]:
# Official pretrained models
!cp /content/gdrive/MyDrive/ML_data/pretrained/quickvc_official/G_1200000.pth ./logs/quickvc
!cp /content/gdrive/MyDrive/ML_data/pretrained/quickvc_official/D_1200000.pth ./logs/quickvc
!cp ./logs/quickvc/G_1200000.pth ./logs/quickvc/quickvc.pth

In [ ]:
# Remove samples
!rm -r ./output/quickvc/*
!ls ./output/quickvc

## Data

In [ ]:
!cp /content/gdrive/MyDrive/ML_data/corpuses/VCTK/vctk2019ds/DS_10283_3443.zip .
!unzip "DS_10283_3443.zip"
!mkdir -p "./dataset/vctk"
!unzip -d "./dataset/vctk" VCTK-Corpus-0.92.zip

## Preprocessing

In [ ]:
!python downsample.py

In [ ]:
%cd dataset
!python encode.py soft vctk-16k vctk-16k
%cd ..

## Training

In [ ]:
!mkdir temp_result

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir "/content/gdrive/MyDrive/ML_results/quickvc_official/default/version_1"

!python train.py -mr "/content/gdrive/MyDrive/ML_results/quickvc_official/default/version_1"

## Inference

In [ ]:
!python convert.py
!ls output/quickvc/

In [ ]:
from pathlib import Path
import librosa
from IPython.display import Audio, display

for p in Path("./output/quickvc").glob('**/*.wav'):
    audio, sr = librosa.load(p)
    display(Audio(audio, rate=sr, normalize=False))
